# Visualizations of sorting model results for dissertation and defense

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp
import zipfile
from glob import glob
import matplotlib.patches as mpatch
import tempfile
import os
import re

In [ ]:
strip_model_prefix = re.compile('^.*FULL_sim_[0-9]{4}-[0-9]{2}-[0-9]{2}_[0-9]{2}_[0-9]{2}_[0-9]{2}_')

In [ ]:
def weighted_percentile(vals, percentiles, weights):
    if len(vals) != len(weights):
        raise ArgumentError("values and weights arrays are not same length!")

    nas = pd.isnull(vals) | pd.isnull(weights)

    nnas = np.sum(nas)
    if nnas > 0:
        warn(f"found {nnas} NAs in data, dropping them")

    vals = vals[~nas]
    weights = weights[~nas]

    weights = weights / np.sum(weights)
    sortIdx = np.argsort(vals)
    vals = vals.iloc[sortIdx]
    weights = weights.iloc[sortIdx]

    cumWeights = np.cumsum(weights)
    if not isinstance(percentiles, np.ndarray):
        percentiles = np.array(percentiles)
    percentiles = percentiles / 100

    # center weights, i.e. put the point value halfway through the weight
    # https://github.com/nudomarinero/wquantiles/blob/master/wquantiles.py
    centeredCumWeights = cumWeights - 0.5 * weights
    return np.interp(percentiles, centeredCumWeights, vals)

In [ ]:
plt.style.use('asu-light')

In [ ]:
pumas = gp.read_file('/Volumes/Pheasant Ridge/IPUMS/pumas/socal_pumas_projected.shp')

In [ ]:
short_names = {
    'existing': 'Existing (model estimation sample)',
    'base': 'Fitted values',
    'rhna': 'RHNA',
    'npv_base': 'Low appreciation',
    'npv_current_appreciation': 'Current appreciation',
    'npv_current_appreciation_hqta': 'Current appreciation (HQTA)',
    'npv_low_opcost_hqta': 'Low operating cost (HQTA)',
    'npv_low_opcost': 'Low operating cost',
    'rhna_nr': 'RHNA (NR)',
    'npv_base_nr': 'Low appreciation (NR)',
    'npv_current_appreciation_nr': 'Current appreciation (NR)',
    'npv_current_appreciation_hqta_nr': 'Current appreciation (HQTA) (NR)',
    'npv_low_opcost_hqta_nr': 'Low operating cost (HQTA) (NR)',
    'npv_low_opcost_nr': 'Low operating cost (NR)'
}

    
long_names = {v: k for k, v in short_names.items()}

In [ ]:
land = gp.read_file('../../sorting/data/ne_10m_land.shp').to_crs(epsg=26911)

roads = pd.concat([gp.read_file(i).to_crs(epsg=26911) for i in glob('../../sorting/data/tl_roads/*.shp')], ignore_index=True)

counties = gp.read_file('../../sorting/data/counties/tl_2019_us_county.shp').to_crs(26911)
counties = counties[(counties.STATEFP == '06') & counties.NAME.isin(['Los Angeles', 'Ventura', 'Orange', 'Riverside', 'San Bernardino', 'Imperial'])]

In [ ]:
f, axs = plt.subplots(4, 2, figsize=(8, 12))
axs = axs.reshape(-1)
axix_for_sname =  [ 'npv_current_appreciation', 'npv_base',
       'npv_low_opcost', 'rhna',
        'npv_current_appreciation_hqta',
    'npv_low_opcost_hqta']

lax = axs[1]
axs = [axs[0], *axs[2:]]
axs[-1].set_axis_off()

for MODEL_FILE in glob('/Volumes/Pheasant Ridge/diss_data/model_output/defense/simulation_*.parquet.zip'):
    sname = strip_model_prefix.sub('', MODEL_FILE).replace('.parquet.zip', '')
    if sname == 'base':
        continue # no change to prices for base values
    ax = axs[axix_for_sname.index(sname)]
    with zipfile.ZipFile(MODEL_FILE) as zin:
        with zin.open('housing.parquet.gz') as hin:
            hsg = pd.read_parquet(hin)

    hsg['pricediff'] = (hsg.new_price - hsg.orig_price) / 12 * 1000
    maxabsdiff = hsg.pricediff.abs().max()

    hsg['htype'] = hsg.index
    hsg[['puma', 'typ', 'age', 'tenure']] = hsg.htype.str.split('_', expand=True)

    renters = hsg[hsg.tenure == 'rent'].copy()
    grpd = renters.groupby('puma')
    rents = pd.DataFrame({
        'orig': grpd.apply(lambda df: np.average(df.orig_price, weights=df.existing_supply)),
        'new': grpd.apply(lambda df: np.average(df.new_price, weights=df.new_supply))
    })

    hsg.pricediff.abs().max()

    hsg['pricediffcat'] = pd.cut(hsg.pricediff, [-4000, -500, -50, 50, 500, 4000]).apply(lambda c: f'${int(c.left)}–${int(c.right)}')

    hsg['pricediffcolor'] = hsg.pricediffcat.replace({
        '$-4000–$-500': '#ca0020',
        '$-500–$-50': '#f4a582',
        '$-50–$50': '#f7f7f7',
        '$50–$500': '#92c5de',
        '$500–$4000': '#0571b0'
    })

    hsg.pricediffcat.unique()

    hsg['htype'] = hsg.index

    # deannualize and convert to unit dollars
    rents['pricediff'] = (rents.new - rents.orig) / 12 * 1000
    print(rents.pricediff.abs().max())

    rents['pricediffcat'] = pd.cut(rents.pricediff, [-1000, -250, -100, 100, 250, 1000]).apply(lambda c: f'${int(c.left)}–${int(c.right)}')

    colors = {
        '$-1000–$-250': '#ca0020',
        '$-250–$-100': '#f4a582',
        '$-100–$100': '#f7f7f7',
        '$100–$250': '#92c5de',
        '$250–$1000': '#0571b0'
    }


    pumas_rent = pumas.merge(rents, left_on='PUMA', right_on='puma', validate='m:1')

    pumas_rent.to_crs(epsg=26911).plot(ax=ax, color=pumas_rent.pricediffcat.replace(colors))
    roads.plot(color='#888888', ax=ax, lw=0.25)
    counties.plot(edgecolor='#000',  facecolor='none', ax=ax, lw=1)
    #water.plot(color='#aaaaaa', ax=ax)
    ax.set_ylim(3.59e6, 3.88e6)
    ax.set_xlim(2.75e5, 7.7e5)
    ax.set_xticks([])
    ax.set_yticks([])

    ax.set_yticks([])
    ax.set_xticks([])
    
    ax.set_xlabel(short_names[sname])


lax.legend(
    [mpatch.Patch(color=c) for c in colors.values()],
    [i.replace('$-', '-$').replace('$', '\\$') for i in colors.keys()],
    loc='center',
    title='Change in average rent',
    framealpha=1,
    fontsize='medium',
    title_fontsize='medium'
)
lax.set_axis_off()

plt.savefig('../../dissertation/fig/sorting/rent_change.png', bbox_inches='tight', dpi=300)

In [ ]:
# plot separately for each, for defense presentation
for MODEL_FILE in glob('/Volumes/Pheasant Ridge/diss_data/model_output/defense/simulation_*.parquet.zip'):
    sname = strip_model_prefix.sub('', MODEL_FILE).replace('.parquet.zip', '')
    if sname == 'base':
        continue # no change to prices for base values
    
    f, ax = plt.subplots()
    
    with zipfile.ZipFile(MODEL_FILE) as zin:
        with zin.open('housing.parquet.gz') as hin:
            hsg = pd.read_parquet(hin)

    hsg['pricediff'] = (hsg.new_price - hsg.orig_price) / 12 * 1000
    maxabsdiff = hsg.pricediff.abs().max()

    hsg['htype'] = hsg.index
    hsg[['puma', 'typ', 'age', 'tenure']] = hsg.htype.str.split('_', expand=True)

    renters = hsg[hsg.tenure == 'rent'].copy()
    grpd = renters.groupby('puma')
    rents = pd.DataFrame({
        'orig': grpd.apply(lambda df: np.average(df.orig_price, weights=df.existing_supply)),
        'new': grpd.apply(lambda df: np.average(df.new_price, weights=df.new_supply))
    })

    hsg.pricediff.abs().max()

    hsg['pricediffcat'] = pd.cut(hsg.pricediff, [-4000, -500, -50, 50, 500, 4000]).apply(lambda c: f'${int(c.left)}–${int(c.right)}')

    hsg['pricediffcolor'] = hsg.pricediffcat.replace({
        '$-4000–$-500': '#ca0020',
        '$-500–$-50': '#f4a582',
        '$-50–$50': '#f7f7f7',
        '$50–$500': '#92c5de',
        '$500–$4000': '#0571b0'
    })

    hsg.pricediffcat.unique()

    hsg['htype'] = hsg.index

    # deannualize and convert to unit dollars
    rents['pricediff'] = (rents.new - rents.orig) / 12 * 1000
    print(rents.pricediff.abs().max())

    rents['pricediffcat'] = pd.cut(rents.pricediff, [-1000, -250, -100, 100, 250, 1000]).apply(lambda c: f'${int(c.left)}–${int(c.right)}')

    colors = {
        '$-1000–$-250': '#ca0020',
        '$-250–$-100': '#f4a582',
        '$-100–$100': '#f7f7f7',
        '$100–$250': '#92c5de',
        '$250–$1000': '#0571b0'
    }


    pumas_rent = pumas.merge(rents, left_on='PUMA', right_on='puma', validate='m:1')

    pumas_rent.to_crs(epsg=26911).plot(ax=ax, color=pumas_rent.pricediffcat.replace(colors))
    roads.plot(color='#888888', ax=ax, lw=0.25)
    counties.plot(edgecolor='#000',  facecolor='none', ax=ax, lw=1)
    #water.plot(color='#aaaaaa', ax=ax)
    ax.set_ylim(3.59e6, 3.88e6)
    ax.set_xlim(2.75e5, 7.7e5)
    ax.set_xticks([])
    ax.set_yticks([])

    ax.set_yticks([])
    ax.set_xticks([])
    
    ax.set_xlabel(short_names[sname])
    plt.savefig(f'../../defense/rent_change_{sname}.png', bbox_inches='tight', dpi=300)

## Compute vehicle ownership

In [ ]:
hh = pd.read_parquet('../data/full_hh.parquet')

In [ ]:
veh_ownership_newres = pd.read_parquet('../model_output/defense/veh_own_new_res.parquet')
veh_ownership = pd.read_parquet('../model_output/defense/veh_own.parquet')

veh_ownership_newres.index = map('{}_nr'.format, veh_ownership_newres.index)

veh_ownership = pd.concat([veh_ownership, veh_ownership_newres])

veh_ownership.dtypes

veh_ownership.loc['existing'] = hh.groupby('vehchoice').hhwt.sum()
veh_ownership['average'] = (veh_ownership['1'] + 2 * veh_ownership['2'] + 3.5016346687714526 * veh_ownership['3+']) / veh_ownership.apply(sum, 1)
veh_ownership[['0', '1', '2', '3+']] = veh_ownership[['0', '1', '2', '3+']].apply(lambda r: r / r.sum() * 100, 1).round(1).astype('str') + '%'
veh_ownership = veh_ownership.rename(index=lambda c: strip_model_prefix.sub('', c))
veh_ownership = veh_ownership.loc[[
'existing', 'base', 'rhna', 'npv_current_appreciation', 'npv_base',        'npv_low_opcost', 'npv_current_appreciation_hqta',

        
    'npv_low_opcost_hqta',
'npv_current_appreciation_nr', 'npv_base_nr',   
       'npv_low_opcost_nr','rhna_nr',
        'npv_current_appreciation_hqta_nr',
    'npv_low_opcost_hqta_nr',
]]
veh_ownership = veh_ownership.rename(index=short_names)
veh_ownership['average'] = veh_ownership.average.round(3)
veh_ownership = veh_ownership.rename(columns={'average': 'Average'})
veh_ownership

In [ ]:
print(veh_ownership.to_latex())

In [ ]:
# bar plots for defense
sub = [i for i in veh_ownership.index if not 'HQTA' in i and not 'NR' in i and not 'Existing' in i]
plt.barh(-np.arange(len(sub)), veh_ownership.loc[sub, 'Average'])
plt.yticks(-np.arange(len(sub)), sub)
plt.xticks([0, 1, 2])
plt.xlabel('Average household vehicle ownership')
plt.savefig('../../defense/car_ownership.pdf', bbox_inches='tight')

In [ ]:
# bar plots for defense
sub = [i for i in veh_ownership.index if not 'HQTA' in i and ('NR' in i or 'Fitted' in i)]
plt.barh(-np.arange(len(sub)), veh_ownership.loc[sub, 'Average'])
plt.yticks(-np.arange(len(sub)), sub)
plt.xticks([0, 1, 2])
plt.xlabel('Average household vehicle ownership')
plt.savefig('../../defense/car_ownership_nr.pdf', bbox_inches='tight')

## Median income by PUMA

In [ ]:
med_inc_puma = (
    pd.read_parquet('../model_output/defense/median_income.parquet')
        .transpose()
        .rename(columns=lambda x: strip_model_prefix.sub('', x))
)

In [ ]:
med_inc_puma

In [ ]:
med_inc_puma['existing'] = hh.groupby('puma').apply(lambda df: weighted_percentile(df.hhincome, 50, df.hhwt)).reindex(med_inc_puma.index)
assert not med_inc_puma.existing.isnull().any()

In [ ]:
med_inc_puma.apply(np.std)

In [ ]:
def rename_cats (cat):
    if not np.isfinite(cat.left):
        return f'≤ ${int(cat.right * 1000):,d}'
    elif not np.isfinite(cat.right):
        return f'> ${int(cat.left * 1000):,d}'
    else:
        return f'\\${int(cat.left * 1000):,d}–\\${int(cat.right * 1000):,d}'

In [ ]:
pumas_inc = pumas.merge(med_inc_puma, left_on='PUMA', right_index=True, validate='m:1').to_crs(epsg=26911)

def plot_inc(col, basecol, title, ax=None, legend=True, cax=None, basemap=True):
    if ax is None:
        f, ax = plt.subplots(figsize=(12, 8))
    pumas_inc.plot(ax=ax, column=pd.cut(pumas_inc[col] - pumas_inc[basecol], [-np.inf, -10, -5, 5, 10, np.inf]).map(rename_cats), cmap='RdBu', legend=legend, cax=cax,
                  legend_kwds={'title': 'Change in median income'})
    if basemap:
        roads.plot(color='#888888', ax=ax, lw=0.15)
        counties.plot(edgecolor='#000',  facecolor='none', ax=ax, lw=1)
        #water.plot(color='#aaaaaa', ax=ax)
    ax.set_ylim(3.59e6, 3.88e6)
    ax.set_xlim(2.75e5, 7.7e5)
    ax.set_xticks([])
    ax.set_yticks([])

    ax.set_yticks([])
    ax.set_xticks([])
    
    ax.set_xlabel(title)

# axs[-1].legend(
#     [mpatch.Patch(color=c) for c in colors.values()],
#     [i.replace('$-', '-$').replace('$', '\\$') for i i`n colors.keys()],
#     loc='center',
#     title='Change in average rent',
#     framealpha=1,
#     fontsize='medium',
#     title_fontsize='medium'
# )
# axs[-1].set_axis_off()

In [ ]:
plot_inc('base', 'existing', None)
plt.savefig('../../dissertation/fig/sorting/median_income_fitted.png', dpi=300, bbox_inches='tight')

In [ ]:
f, axs = plt.subplots(4, 2, figsize=(8, 12))
axs = axs.reshape(-1)
axix_for_sname =  [ 'npv_current_appreciation', 'npv_base',
       'npv_low_opcost', 'rhna',
        'npv_current_appreciation_hqta',
    'npv_low_opcost_hqta']

axs[1].set_axis_off()
axs[-1].set_axis_off()

for sname, ax in zip(axix_for_sname, [axs[0], *axs[2:-1]]):
    plot_inc(sname, 'base', short_names[sname], ax=ax, legend=False)
    
# hack to make legend appear, cax does not work for categoricals it seems
plot_inc('base', 'existing', '', ax=axs[1], basemap=False)
# pumas_inc.plot(ax=axs[1], column='legcol', cmap='RdBu', legend=True,
#                   legend_kwds={'title': 'Change in median income'})
pumas_inc.plot(ax=axs[1], color='white', edgecolor='white')
plt.savefig('../../dissertation/fig/sorting/median_income.png', dpi=300, bbox_inches='tight')

### Plot each scenario separately for the defense

In [ ]:
for scenario in axix_for_sname:
    plot_inc(scenario, 'base', short_names[scenario])
    plt.savefig(f'../../defense/med_inc_{scenario}.pdf', bbox_inches='tight')

## Income IQR

In [ ]:
iqr_inc_puma = (
        pd.read_parquet('../model_output/defense/iqr_income.parquet')
        .transpose()
        .rename(columns=lambda x: strip_model_prefix.sub('', x))
)

In [ ]:
iqr_inc_puma.head()

In [ ]:
pumas_iqr = pumas.merge(iqr_inc_puma, left_on='PUMA', right_index=True, validate='m:1').to_crs(epsg=26911)

def plot_iqr(col, basecol, title, ax=None, legend=True, cax=None, basemap=True):
    if ax is None:
        f, ax = plt.subplots(figsize=(12, 8))
    pumas_iqr.plot(ax=ax, column=pd.cut(pumas_iqr[col] - pumas_iqr[basecol], [-np.inf, -5, -1, 1, 5, np.inf]).map(rename_cats), cmap='RdBu', legend=legend, cax=cax,
                  legend_kwds={'title': 'Change income interquartile range'})
    if basemap:
        roads.plot(color='#888888', ax=ax, lw=0.15)
        counties.plot(edgecolor='#000',  facecolor='none', ax=ax, lw=1)
        #water.plot(color='#aaaaaa', ax=ax)
    ax.set_ylim(3.59e6, 3.88e6)
    ax.set_xlim(2.75e5, 7.7e5)
    ax.set_xticks([])
    ax.set_yticks([])

    ax.set_yticks([])
    ax.set_xticks([])
    
    ax.set_xlabel(title)

In [ ]:
f, axs = plt.subplots(4, 2, figsize=(8, 12))
axs = axs.reshape(-1)
axix_for_sname =  [ 'npv_current_appreciation', 'npv_base',
       'npv_low_opcost', 'rhna',
        'npv_current_appreciation_hqta',
    'npv_low_opcost_hqta']

axs[1].set_axis_off()
axs[-1].set_axis_off()

for sname, ax in zip(axix_for_sname, [axs[0], *axs[2:-1]]):
    plot_iqr(sname, 'base', short_names[sname], ax=ax, legend=False)
    
# hack to make legend appear, cax does not work for categoricals it seems
plot_iqr('base', 'base', '', ax=axs[1], basemap=False)
# pumas_inc.plot(ax=axs[1], column='legcol', cmap='RdBu', legend=True,
#                   legend_kwds={'title': 'Change in median income'})
pumas_iqr.plot(ax=axs[1], color='white', edgecolor='white')
plt.savefig('../../dissertation/fig/sorting/iqr_income.png', dpi=300, bbox_inches='tight')

## Income of new building residents

In [ ]:
pumas_ninc = pumas.merge(med_inc_puma, left_on='PUMA', right_index=True, validate='m:1').to_crs(epsg=26911)

def plot_ninc(col, title, ax=None, legend=True, cax=None, basemap=True):
    if ax is None:
        f, ax = plt.subplots(figsize=(12, 8))
    pumas_ninc.plot(ax=ax, column=pd.cut(pumas_ninc[col], [-np.inf, 35, 50, 75, 100, np.inf]).map(rename_cats), cmap='Blues', legend=legend, cax=cax,
                  legend_kwds={'title': 'Median income'})
    if basemap:
        roads.plot(color='#888888', ax=ax, lw=0.15)
        counties.plot(edgecolor='#000',  facecolor='none', ax=ax, lw=1)
        #water.plot(color='#aaaaaa', ax=ax)
    ax.set_ylim(3.59e6, 3.88e6)
    ax.set_xlim(2.75e5, 7.7e5)
    ax.set_xticks([])
    ax.set_yticks([])

    ax.set_yticks([])
    ax.set_xticks([])
    
    ax.set_xlabel(title)

# axs[-1].legend(
#     [mpatch.Patch(color=c) for c in colors.values()],
#     [i.replace('$-', '-$').replace('$', '\\$') for i i`n colors.keys()],
#     loc='center',
#     title='Change in average rent',
#     framealpha=1,
#     fontsize='medium',
#     title_fontsize='medium'
# )
# axs[-1].set_axis_off()

In [ ]:
f, axs = plt.subplots(4, 2, figsize=(8, 12))
axs = axs.reshape(-1)
axix_for_sname =  [ 'npv_current_appreciation', 'npv_base',
       'npv_low_opcost', 'rhna',
        'npv_current_appreciation_hqta',
    'npv_low_opcost_hqta']

axs[1].set_axis_off()
axs[-1].set_axis_off()

for sname, ax in zip(axix_for_sname, [axs[0], *axs[2:-1]]):
    plot_ninc(sname, short_names[sname], ax=ax, legend=False)
    
# hack to make legend appear, cax does not work for categoricals it seems
plot_ninc(sname, '', ax=axs[1], basemap=False)
# pumas_inc.plot(ax=axs[1], column='legcol', cmap='RdBu', legend=True,
#                   legend_kwds={'title': 'Change in median income'})
pumas_ninc.plot(ax=axs[1], color='white', edgecolor='white')
plt.savefig('../../dissertation/fig/sorting/new_bldg_income.png', dpi=300, bbox_inches='tight')

In [ ]:
axix_for_sname =  [ 'npv_current_appreciation', 'npv_base',
       'npv_low_opcost', 'rhna',
        'npv_current_appreciation_hqta',
    'npv_low_opcost_hqta']

for sname in axix_for_sname:
    plot_ninc(sname, short_names[sname], legend=True)
    
    plt.savefig(f'../../dissertation/fig/sorting/new_bldg_income_{sname}.png', dpi=300, bbox_inches='tight')